# Imports

In [1]:
# Import standard libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# API
import requests
import snowflake.connector

# Date
import datetime as dt
import time
from datetime import timedelta

In [2]:
silver = pd.read_csv('Acuity.csv')
acuity = pd.read_csv('acuity_intake_source_2021.csv')

sub_20192 = pd.read_csv('2019-2.csv')
sub_2020 = pd.read_csv('2020.csv')
sub_2021 = pd.read_csv('2021.csv')

## Acuity

In [6]:
acuity.shape

(1637, 18)

In [8]:
acuity.Campaign.isnull().sum()

591

In [4]:
acuity.Campaign.value_counts()

checkoutsuccess                   434
hmapage                           186
ndpage                             94
cx                                 87
emaildrip                          81
hmafollowup                        74
trackingpage                       38
brightback                         31
socialdm                            8
dr_hma                              4
dr_consult                          4
lcm_210127-newyear-nonsubw-20       2
lcm_210127-newyear-nonsub-b-15      1
lcm_welcome_M_D21                   1
lcm_210127-newyear-nonsubw-15       1
Name: Campaign, dtype: int64

In [24]:
# Change dates to datetime Dtype
silver['APPOINTMENT_BODY_DATE'] = pd.to_datetime(silver['APPOINTMENT_BODY_DATE'])

# Drop unneeded columns
silver.drop(columns=['EVENT_TS', 'EVENT_TS_UTC', 'ANONYMOUS_ID','APPOINTMENT_BODY_FIRST_NAME',
       'APPOINTMENT_BODY_DATETIME_CREATED', 'APPOINTMENT_BODY_CALENDAR_ID','APPOINTMENT_BODY_CALENDAR_TIMEZONE',\
                    'APPOINTMENT_BODY_TIMEZONE', 'APPOINTMENT_BODY_LAST_NAME',\
                    'APPOINTMENT_BODY_NO_SHOW', \
                    'APPOINTMENT_TYPE_ID'], inplace=True)

silver['EVENT'].value_counts()

appointment_changed        19261
appointment_scheduled       5664
appointment_rescheduled      774
appointment_canceled         477
Name: EVENT, dtype: int64

In [11]:
clean = silver.groupby('EMAIL').agg({"APPOINTMENT_BODY_DATE":'min',\
                                'APPOINTMENT_BODY_TYPE':'last',\
                                'APPOINTMENT_BODY_CALENDAR':'last','APPOINTMENT_BODY_ID':'min'\
                               })
clean.shape

(4958, 4)

In [12]:
acuity = pd.merge(intake,clean, how='left',left_on='Appointment ID', right_on='APPOINTMENT_BODY_ID',copy=False)
# acuity.info()

In [13]:
acuity.Email.nunique()

265

## SUB

In [14]:
sub_2019=pd.merge(sub_20191,sub_20192,how='outer')
print('total',sub_2019.shape)
print('1',sub_20191.shape)
print('2',sub_20192.shape)

# Change dates to datetime
sub_2017['FIRST_ORDER_AT'] = pd.to_datetime(sub_2017['FIRST_ORDER_AT'])
sub_2018['FIRST_ORDER_AT'] = pd.to_datetime(sub_2018['FIRST_ORDER_AT'])
sub_2019['FIRST_ORDER_AT'] = pd.to_datetime(sub_2019['FIRST_ORDER_AT'])
sub_2020['FIRST_ORDER_AT'] = pd.to_datetime(sub_2020['FIRST_ORDER_AT'])
sub_2021['FIRST_ORDER_AT'] = pd.to_datetime(sub_2021['FIRST_ORDER_AT'])

total (1147363, 7)
1 (506714, 7)
2 (640649, 7)


In [15]:
# Aggregate
df17 = sub_2017.groupby('EMAIL').agg({"FIRST_ORDER_AT":'min',\
                                'TOTAL_ORDER_VALUE':'mean',\
                                'TOTAL_ORDERS':'mean'\
                                ,'ITEM_GENDER':'last',\
                                'ITEM_SUB_TYPE':'last',\
                                'ITEM_PRODUCT_CORE_CATEGORY':'last'\
                               })
df17.shape

(3792, 6)

In [16]:
# Aggregate
df18 = sub_2018.groupby('EMAIL').agg({"FIRST_ORDER_AT":'min',\
                                'TOTAL_ORDER_VALUE':'mean',\
                                'TOTAL_ORDERS':'mean'\
                                ,'ITEM_GENDER':'last',\
                                'ITEM_SUB_TYPE':'last',\
                                'ITEM_PRODUCT_CORE_CATEGORY':'last'\
                               })
df18.shape

(18970, 6)

In [17]:
# Aggregate
df19 = sub_2019.groupby('EMAIL').agg({"FIRST_ORDER_AT":'min',\
                                'TOTAL_ORDER_VALUE':'mean',\
                                'TOTAL_ORDERS':'mean'\
                                ,'ITEM_GENDER':'last',\
                                'ITEM_SUB_TYPE':'last',\
                                'ITEM_PRODUCT_CORE_CATEGORY':'last'\
                               })
df19.shape

(49744, 6)

In [18]:
# Aggregate
df20 = sub_2020.groupby('EMAIL').agg({"FIRST_ORDER_AT":'min',\
                                'TOTAL_ORDER_VALUE':'mean',\
                                'TOTAL_ORDERS':'mean'\
                                ,'ITEM_GENDER':'last',\
                                'ITEM_SUB_TYPE':'last',\
                                'ITEM_PRODUCT_CORE_CATEGORY':'last'\
                               })
df20.shape

(82292, 6)

In [19]:
# Aggregate
df21 = sub_2021.groupby('EMAIL').agg({"FIRST_ORDER_AT":'min',\
                                'TOTAL_ORDER_VALUE':'mean',\
                                'TOTAL_ORDERS':'mean'\
                                ,'ITEM_GENDER':'last',\
                                'ITEM_SUB_TYPE':'last',\
                                'ITEM_PRODUCT_CORE_CATEGORY':'last'\
                               })
df21.shape

(19819, 6)

In [20]:
sub1=pd.merge(sub_2019,sub_2018,how='outer')
sub2=pd.merge(sub_2020,sub_2021,how='outer')
sub4=pd.merge(sub1,sub2,how='outer')
sub3=pd.merge(sub4,sub_2017,how='outer')

In [21]:
sub3 = sub3.groupby('EMAIL').agg({"FIRST_ORDER_AT":'min',\
                                'TOTAL_ORDER_VALUE':'mean',\
                                'TOTAL_ORDERS':'mean'\
                                ,'ITEM_GENDER':'last',\
                                'ITEM_SUB_TYPE':'last',\
                                'ITEM_PRODUCT_CORE_CATEGORY':'last'\
                               })
sub3.shape

(174354, 6)

## Merge

In [28]:
sub = pd.merge(acuity,sub3, how='left',left_on='Email',right_on='EMAIL',copy=False)
sub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 265 entries, 0 to 264
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Start Time                  265 non-null    datetime64[ns]
 1   End Time                    265 non-null    object        
 2   First Name                  265 non-null    object        
 3   Last Name                   265 non-null    object        
 4   Phone                       265 non-null    object        
 5   Email                       265 non-null    object        
 6   Type                        265 non-null    object        
 7   Calendar                    265 non-null    object        
 8   Appointment Price           265 non-null    float64       
 9   Paid?                       265 non-null    object        
 10  Amount Paid Online          265 non-null    float64       
 11  Date Scheduled              265 non-null    object        

In [29]:
# clean up item sub
sub.loc[sub['ITEM_SUB_TYPE'] =='GrowthPlan,GrowthPlan']= 'GrowthPlan'
sub.loc[sub['ITEM_SUB_TYPE'] =='GrowthPlan 3,GrowthPlan 3']= 'GrowthPlan 3'
sub.loc[sub['ITEM_SUB_TYPE'] =='GrowthPlan,GrowthPlan 3']= 'GrowthPlan'
sub.loc[sub['ITEM_SUB_TYPE'] =='GrowthPlan 2,GrowthPlan']= 'GrowthPlan 2'
sub.loc[sub['ITEM_SUB_TYPE'] =='GrowthPlan 3,GrowthPlan 2']= 'GrowthPlan 3'
sub.loc[sub['ITEM_SUB_TYPE'] =='GrowthPlan,GrowthPlan 2']= 'GrowthPlan'
sub.loc[sub['ITEM_SUB_TYPE'] =='GrowthPlan 3,GrowthPlan']= 'GrowthPlan 3'
sub.loc[sub['ITEM_SUB_TYPE'] =='GrowthPlan 2,GrowthPlan 2']= 'GrowthPlan 2'
sub.loc[sub['ITEM_SUB_TYPE'] =='GrowthPlan 3,Non-Sub']= 'GrowthPlan 3'

sub['ITEM_SUB_TYPE'].value_counts()

GrowthPlan      89
GrowthPlan 3    86
GrowthPlan 2    42
Name: ITEM_SUB_TYPE, dtype: int64

In [30]:
sub.loc[sub['ITEM_PRODUCT_CORE_CATEGORY'] =='Men,Men']= 'Men'
sub.loc[sub['ITEM_PRODUCT_CORE_CATEGORY'] =='GrowthPlan']= 'Balance'
sub.loc[sub['ITEM_PRODUCT_CORE_CATEGORY'] =='GrowthPlan 2']= 'Balance'
sub.loc[sub['ITEM_PRODUCT_CORE_CATEGORY'] =='GrowthPlan 3']= 'Balance'

sub['ITEM_PRODUCT_CORE_CATEGORY'].value_counts()

Women      115
Balance     66
Men         36
Name: ITEM_PRODUCT_CORE_CATEGORY, dtype: int64

In [31]:
# Export data
sub.to_csv('intake_sub.csv')
sub.describe()

,Start Time,End Time,First Name,Last Name,Phone,Email,Type,Calendar,Appointment Price,Paid?,...,APPOINTMENT_BODY_DATE,APPOINTMENT_BODY_TYPE,APPOINTMENT_BODY_CALENDAR,APPOINTMENT_BODY_ID,FIRST_ORDER_AT,TOTAL_ORDER_VALUE,TOTAL_ORDERS,ITEM_GENDER,ITEM_SUB_TYPE,ITEM_PRODUCT_CORE_CATEGORY
count,265,265,265,265,265,265,265,265,265.0,265,...,205,205,205,205,217,217.0,217.0,217,217,217
unique,253,257,221,261,264,264,3,10,2.0,2,...,35,4,10,204,216,150.0,26.0,3,4,3
top,2021-01-27 13:00:00,"February 10, 2021 9:25 am",Nicole,Thompson,Balance,Balance,Hair Consultation,"Dr. Ariana Zastrow, ND",0.0,no,...,2021-02-16 00:00:00,Hair Consultation,"Dr. Ariana Zastrow, ND",Balance,Balance,99.0,2.0,Women,GrowthPlan,Women
freq,3,2,3,2,2,2,185,62,263.0,263,...,13,169,53,2,2,6.0,28.0,179,87,115
